In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2


In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)


cpu


In [5]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [6]:
train_path='C:/Users/Z004CCDD/Downloads/selectedclasses/train/'
test_path='C:/Users/Z004CCDD/Downloads/selectedclasses/test'
pred_path='C:/Users/Z004CCDD/Downloads/selectedclasses/pred'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [7]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [8]:
print(classes)

['airplane', 'alarm_clock', 'apple', 'bicycle', 'car', 'chair', 'flower', 'mushroom', 'sailboat', 'umbrella']


In [9]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [10]:
model=ConvNet(num_classes=10).to(device)

In [11]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [12]:
num_epochs=10

In [13]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [14]:
print(train_count,test_count)

720 180


In [15]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(18.5108) Train Accuracy: 0.9333333333333333 Test Accuracy: 0.31666666666666665
Epoch: 1 Train Loss: tensor(5.3374) Train Accuracy: 1.2694444444444444 Test Accuracy: 0.4166666666666667
Epoch: 2 Train Loss: tensor(2.2470) Train Accuracy: 1.4166666666666667 Test Accuracy: 0.5722222222222222
Epoch: 3 Train Loss: tensor(1.2367) Train Accuracy: 1.5347222222222223 Test Accuracy: 0.6
Epoch: 4 Train Loss: tensor(0.8955) Train Accuracy: 1.5847222222222221 Test Accuracy: 0.6222222222222222
Epoch: 5 Train Loss: tensor(0.6284) Train Accuracy: 1.5986111111111112 Test Accuracy: 0.6111111111111112
Epoch: 6 Train Loss: tensor(0.3436) Train Accuracy: 1.6541666666666666 Test Accuracy: 0.6166666666666667
Epoch: 7 Train Loss: tensor(0.1981) Train Accuracy: 1.6819444444444445 Test Accuracy: 0.6611111111111111
Epoch: 8 Train Loss: tensor(0.2342) Train Accuracy: 1.7 Test Accuracy: 0.6944444444444444
Epoch: 9 Train Loss: tensor(0.2587) Train Accuracy: 1.6888888888888889 Test Accurac

In [16]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [17]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [18]:
images_path=glob.glob(pred_path+'/*.jpg')

In [24]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [25]:
pred_dict

{'pred\\airplane0.jpg': 'airplane',
 'pred\\airplane1.jpg': 'airplane',
 'pred\\airplane10.jpg': 'airplane',
 'pred\\airplane11.jpg': 'airplane',
 'pred\\airplane12.jpg': 'sailboat',
 'pred\\airplane13.jpg': 'airplane',
 'pred\\airplane14.jpg': 'car',
 'pred\\airplane15.jpg': 'airplane',
 'pred\\airplane16.jpg': 'sailboat',
 'pred\\airplane17.jpg': 'alarm_clock',
 'pred\\airplane18.jpg': 'sailboat',
 'pred\\airplane19.jpg': 'airplane',
 'pred\\airplane2.jpg': 'airplane',
 'pred\\airplane3.jpg': 'bicycle',
 'pred\\airplane4.jpg': 'sailboat',
 'pred\\airplane5.jpg': 'sailboat',
 'pred\\airplane6.jpg': 'sailboat',
 'pred\\airplane7.jpg': 'airplane',
 'pred\\airplane8.jpg': 'airplane',
 'pred\\airplane9.jpg': 'chair',
 'pred\\alarm_clock0.jpg': 'alarm_clock',
 'pred\\alarm_clock1.jpg': 'mushroom',
 'pred\\alarm_clock10.jpg': 'alarm_clock',
 'pred\\alarm_clock11.jpg': 'alarm_clock',
 'pred\\alarm_clock12.jpg': 'alarm_clock',
 'pred\\alarm_clock13.jpg': 'alarm_clock',
 'pred\\alarm_clock14.j

In [29]:
user_input='airplane'

In [36]:
def value(i,transformer,user_input):
    if(prediction(i,transformer)==user_input):
        return user_input
    else:
        return 'user_input'

In [37]:

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=value(i,transformer,user_input)

In [38]:
 pred_dict

{'pred\\airplane0.jpg': 'airplane',
 'pred\\airplane1.jpg': 'airplane',
 'pred\\airplane10.jpg': 'airplane',
 'pred\\airplane11.jpg': 'airplane',
 'pred\\airplane12.jpg': 'user_input',
 'pred\\airplane13.jpg': 'airplane',
 'pred\\airplane14.jpg': 'user_input',
 'pred\\airplane15.jpg': 'airplane',
 'pred\\airplane16.jpg': 'user_input',
 'pred\\airplane17.jpg': 'user_input',
 'pred\\airplane18.jpg': 'airplane',
 'pred\\airplane19.jpg': 'airplane',
 'pred\\airplane2.jpg': 'airplane',
 'pred\\airplane3.jpg': 'user_input',
 'pred\\airplane4.jpg': 'user_input',
 'pred\\airplane5.jpg': 'user_input',
 'pred\\airplane6.jpg': 'user_input',
 'pred\\airplane7.jpg': 'airplane',
 'pred\\airplane8.jpg': 'airplane',
 'pred\\airplane9.jpg': 'user_input',
 'pred\\alarm_clock0.jpg': 'user_input',
 'pred\\alarm_clock1.jpg': 'user_input',
 'pred\\alarm_clock10.jpg': 'user_input',
 'pred\\alarm_clock11.jpg': 'user_input',
 'pred\\alarm_clock12.jpg': 'user_input',
 'pred\\alarm_clock13.jpg': 'user_input',
 '